In [5]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from hyperparameter_tuning import random_search
from feature_builder import process_dataset
import nltk

In [6]:
x_train = pd.read_csv('train.csv')
y_train=x_train[['id','target']]

In [7]:
x_train_features = process_dataset(x_train)

Percentage of words covered in the embeddings = 0.6336399642263958


In [8]:
x_train_feat, x_test_feat, y_train_feat, y_test_feat = train_test_split(x_train_features, y_train, test_size = .33, random_state = 17)

In [9]:
catboost = CatBoostClassifier()


catboost.fit(x_train_feat, y_train_feat['target'])

Learning rate set to 0.020657
0:	learn: 0.6860789	total: 692ms	remaining: 11m 31s
1:	learn: 0.6775309	total: 748ms	remaining: 6m 13s
2:	learn: 0.6702710	total: 793ms	remaining: 4m 23s
3:	learn: 0.6630542	total: 839ms	remaining: 3m 28s
4:	learn: 0.6561331	total: 890ms	remaining: 2m 57s
5:	learn: 0.6494344	total: 953ms	remaining: 2m 37s
6:	learn: 0.6436250	total: 991ms	remaining: 2m 20s
7:	learn: 0.6375557	total: 1.04s	remaining: 2m 8s
8:	learn: 0.6312698	total: 1.08s	remaining: 1m 58s
9:	learn: 0.6252765	total: 1.14s	remaining: 1m 52s
10:	learn: 0.6194385	total: 1.18s	remaining: 1m 45s
11:	learn: 0.6147140	total: 1.23s	remaining: 1m 40s
12:	learn: 0.6091953	total: 1.27s	remaining: 1m 36s
13:	learn: 0.6038623	total: 1.32s	remaining: 1m 33s
14:	learn: 0.5991868	total: 1.37s	remaining: 1m 29s
15:	learn: 0.5948421	total: 1.41s	remaining: 1m 26s
16:	learn: 0.5899229	total: 1.46s	remaining: 1m 24s
17:	learn: 0.5849038	total: 1.52s	remaining: 1m 23s
18:	learn: 0.5809658	total: 1.56s	remaining:

162:	learn: 0.3976959	total: 8.91s	remaining: 45.8s
163:	learn: 0.3972017	total: 8.96s	remaining: 45.7s
164:	learn: 0.3966573	total: 9s	remaining: 45.6s
165:	learn: 0.3960509	total: 9.04s	remaining: 45.4s
166:	learn: 0.3954762	total: 9.12s	remaining: 45.5s
167:	learn: 0.3949732	total: 9.17s	remaining: 45.4s
168:	learn: 0.3945496	total: 9.22s	remaining: 45.3s
169:	learn: 0.3941057	total: 9.26s	remaining: 45.2s
170:	learn: 0.3936372	total: 9.31s	remaining: 45.1s
171:	learn: 0.3931107	total: 9.36s	remaining: 45.1s
172:	learn: 0.3925955	total: 9.41s	remaining: 45s
173:	learn: 0.3920744	total: 9.45s	remaining: 44.8s
174:	learn: 0.3915167	total: 9.49s	remaining: 44.8s
175:	learn: 0.3909081	total: 9.55s	remaining: 44.7s
176:	learn: 0.3904660	total: 9.6s	remaining: 44.6s
177:	learn: 0.3900987	total: 9.65s	remaining: 44.6s
178:	learn: 0.3896664	total: 9.71s	remaining: 44.5s
179:	learn: 0.3892976	total: 9.76s	remaining: 44.4s
180:	learn: 0.3888674	total: 9.81s	remaining: 44.4s
181:	learn: 0.3884

324:	learn: 0.3358317	total: 17s	remaining: 35.4s
325:	learn: 0.3355605	total: 17.1s	remaining: 35.3s
326:	learn: 0.3352271	total: 17.1s	remaining: 35.3s
327:	learn: 0.3348213	total: 17.2s	remaining: 35.2s
328:	learn: 0.3344034	total: 17.2s	remaining: 35.1s
329:	learn: 0.3341143	total: 17.3s	remaining: 35.1s
330:	learn: 0.3338156	total: 17.3s	remaining: 35s
331:	learn: 0.3334828	total: 17.4s	remaining: 35s
332:	learn: 0.3332345	total: 17.4s	remaining: 34.9s
333:	learn: 0.3328938	total: 17.5s	remaining: 34.8s
334:	learn: 0.3325548	total: 17.5s	remaining: 34.8s
335:	learn: 0.3323006	total: 17.6s	remaining: 34.7s
336:	learn: 0.3319930	total: 17.6s	remaining: 34.6s
337:	learn: 0.3316553	total: 17.7s	remaining: 34.6s
338:	learn: 0.3313844	total: 17.7s	remaining: 34.5s
339:	learn: 0.3311073	total: 17.8s	remaining: 34.5s
340:	learn: 0.3307541	total: 17.8s	remaining: 34.4s
341:	learn: 0.3304614	total: 17.8s	remaining: 34.3s
342:	learn: 0.3301994	total: 17.9s	remaining: 34.3s
343:	learn: 0.3299

485:	learn: 0.2843487	total: 25.3s	remaining: 26.8s
486:	learn: 0.2840237	total: 25.4s	remaining: 26.7s
487:	learn: 0.2836515	total: 25.4s	remaining: 26.7s
488:	learn: 0.2833467	total: 25.5s	remaining: 26.6s
489:	learn: 0.2831958	total: 25.5s	remaining: 26.6s
490:	learn: 0.2829996	total: 25.6s	remaining: 26.5s
491:	learn: 0.2827002	total: 25.6s	remaining: 26.4s
492:	learn: 0.2823613	total: 25.7s	remaining: 26.4s
493:	learn: 0.2819982	total: 25.7s	remaining: 26.3s
494:	learn: 0.2817378	total: 25.8s	remaining: 26.3s
495:	learn: 0.2813680	total: 25.8s	remaining: 26.2s
496:	learn: 0.2809919	total: 25.9s	remaining: 26.2s
497:	learn: 0.2806778	total: 25.9s	remaining: 26.1s
498:	learn: 0.2802872	total: 26s	remaining: 26.1s
499:	learn: 0.2798321	total: 26s	remaining: 26s
500:	learn: 0.2795810	total: 26.1s	remaining: 26s
501:	learn: 0.2792145	total: 26.1s	remaining: 25.9s
502:	learn: 0.2788976	total: 26.2s	remaining: 25.9s
503:	learn: 0.2786443	total: 26.2s	remaining: 25.8s
504:	learn: 0.278331

647:	learn: 0.2358529	total: 33.5s	remaining: 18.2s
648:	learn: 0.2356005	total: 33.6s	remaining: 18.2s
649:	learn: 0.2353447	total: 33.6s	remaining: 18.1s
650:	learn: 0.2350794	total: 33.7s	remaining: 18.1s
651:	learn: 0.2347607	total: 33.7s	remaining: 18s
652:	learn: 0.2345507	total: 33.8s	remaining: 18s
653:	learn: 0.2342301	total: 33.8s	remaining: 17.9s
654:	learn: 0.2339098	total: 33.9s	remaining: 17.8s
655:	learn: 0.2335738	total: 33.9s	remaining: 17.8s
656:	learn: 0.2332657	total: 34s	remaining: 17.7s
657:	learn: 0.2330611	total: 34s	remaining: 17.7s
658:	learn: 0.2327407	total: 34.1s	remaining: 17.6s
659:	learn: 0.2326414	total: 34.1s	remaining: 17.6s
660:	learn: 0.2325068	total: 34.2s	remaining: 17.5s
661:	learn: 0.2322484	total: 34.2s	remaining: 17.5s
662:	learn: 0.2319701	total: 34.3s	remaining: 17.4s
663:	learn: 0.2316815	total: 34.3s	remaining: 17.4s
664:	learn: 0.2314258	total: 34.4s	remaining: 17.3s
665:	learn: 0.2312081	total: 34.4s	remaining: 17.3s
666:	learn: 0.230982

810:	learn: 0.1972649	total: 41.8s	remaining: 9.74s
811:	learn: 0.1970790	total: 41.9s	remaining: 9.69s
812:	learn: 0.1969687	total: 41.9s	remaining: 9.64s
813:	learn: 0.1967597	total: 42s	remaining: 9.59s
814:	learn: 0.1965155	total: 42s	remaining: 9.54s
815:	learn: 0.1962489	total: 42.1s	remaining: 9.48s
816:	learn: 0.1960314	total: 42.1s	remaining: 9.43s
817:	learn: 0.1958102	total: 42.2s	remaining: 9.38s
818:	learn: 0.1955938	total: 42.2s	remaining: 9.32s
819:	learn: 0.1953324	total: 42.2s	remaining: 9.27s
820:	learn: 0.1950507	total: 42.3s	remaining: 9.22s
821:	learn: 0.1948067	total: 42.3s	remaining: 9.17s
822:	learn: 0.1946626	total: 42.4s	remaining: 9.12s
823:	learn: 0.1944625	total: 42.4s	remaining: 9.07s
824:	learn: 0.1942326	total: 42.5s	remaining: 9.01s
825:	learn: 0.1939710	total: 42.5s	remaining: 8.96s
826:	learn: 0.1937890	total: 42.6s	remaining: 8.91s
827:	learn: 0.1935517	total: 42.6s	remaining: 8.86s
828:	learn: 0.1933228	total: 42.7s	remaining: 8.8s
829:	learn: 0.193

970:	learn: 0.1663129	total: 49.7s	remaining: 1.48s
971:	learn: 0.1661668	total: 49.8s	remaining: 1.43s
972:	learn: 0.1661009	total: 49.8s	remaining: 1.38s
973:	learn: 0.1659163	total: 49.9s	remaining: 1.33s
974:	learn: 0.1656865	total: 49.9s	remaining: 1.28s
975:	learn: 0.1655194	total: 50s	remaining: 1.23s
976:	learn: 0.1653160	total: 50s	remaining: 1.18s
977:	learn: 0.1652362	total: 50.1s	remaining: 1.13s
978:	learn: 0.1650793	total: 50.1s	remaining: 1.07s
979:	learn: 0.1648637	total: 50.2s	remaining: 1.02s
980:	learn: 0.1646513	total: 50.2s	remaining: 972ms
981:	learn: 0.1644749	total: 50.3s	remaining: 921ms
982:	learn: 0.1642515	total: 50.3s	remaining: 870ms
983:	learn: 0.1641080	total: 50.4s	remaining: 819ms
984:	learn: 0.1639667	total: 50.5s	remaining: 768ms
985:	learn: 0.1637911	total: 50.5s	remaining: 717ms
986:	learn: 0.1636081	total: 50.6s	remaining: 666ms
987:	learn: 0.1634177	total: 50.6s	remaining: 615ms
988:	learn: 0.1632267	total: 50.7s	remaining: 563ms
989:	learn: 0.16

In [10]:
predictions_features = catboost.predict(x_test_feat)
accuracy_score(y_test_feat['target'], predictions_features)

0.8165539196179865

In [11]:
f1_score(y_test_feat['target'], predictions_features)

0.7752315943442224